In [1]:
import requests, json, time, csv
from plotly.io import write_html
import plotly.express as px
import pandas as pd

In [8]:
def map(API_KEY, company_name):
    if type(company_name) != str:
        raise TypeError
    MAX_NUM_RECORDS = -1

    # NO CHANGES NEEDED BELOW HERE
    PDL_URL = "https://api.peopledatalabs.com/v5/person/search"
    request_header = {
        "Content-Type": "application/json",
        "X-api-key": API_KEY
    }

    ES_QUERY = {
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "job_company_name": company_name
                        }
                    }
                ]
            }
        }
    }
    
    num_records_to_request = 100
    params = {
        "dataset": "street_address",
        "query": json.dumps(ES_QUERY),
        "size": num_records_to_request,
        "pretty": True
    }
    
    # Pull all results in multiple batches
    batch = 1
    all_records = []
    start_time = time.time()
    while batch == 1 or params["scroll_token"]:
        if MAX_NUM_RECORDS != -1:
            # Update num_records_to_request
            # Compute the number of records left to pull
            num_records_to_request = MAX_NUM_RECORDS - len(all_records)
            # Clamp this number between 0 and 100
            num_records_to_request = max(0, min(num_records_to_request, 100))

        if num_records_to_request == 0:
            break

        params["size"] = num_records_to_request
        response = requests.get(PDL_URL, headers=request_header, params=params).json()

        if batch == 1:
            print(f"{response['total']} available records in this search")

        all_records.extend(response.get("data", []))
        params["scroll_token"] = response.get("scroll_token")
        print(f"Retrieved {len(response.get('data', []))} records in batch {batch}")
        batch += 1

        if params["scroll_token"]:
            time.sleep(6)   # avoid hitting rate limit thresholds
    
    end_time = time.time()
    runtime = end_time - start_time

    print(f"Successfully recovered {len(all_records)} profiles in "
          f"{batch} batches [{runtime} seconds]")


    def save_profiles_to_csv(profiles, filename, fields=[], delim=","):
        """Save profiles to csv (utility function)"""

        # Define header fields
        if fields == [] and len(profiles) > 0:
            fields = profiles[0].keys()
            
        with open(filename, "w", encoding="utf-8") as csvfile:
            # Write csv file
            writer = csv.writer(csvfile, delimiter=delim)

            # Write Header:
            writer.writerow(fields)

            count = 0
            for profile in profiles:
                # Write Body:
                writer.writerow([profile[field] for field in fields])
                count += 1
                print(f"Wrote {count} lines to: '{filename}'")
                
    # Use utility function to save profiles to csv
    csv_header_fields = ['job_company_location_geo', 'job_company_location_street_address',
                         'job_company_location_address_line_2',
                         'location_street_address', 'location_address_line_2', 'location_geo']
    csv_filename = company_name + "_employees.csv"
    save_profiles_to_csv(all_records, csv_filename, csv_header_fields)
    df = pd.read_csv(csv_filename)
    df['lat'] = df['location_geo'].apply(lambda s: float(s.split(',')[0]))
    df['lon'] = df['location_geo'].apply(lambda s: float(s.split(',')[1]))
    # used Mapbox for map with streets
    token_name = 'pk.eyJ1IjoidHJhY3ljaGFybGVzMTA4IiwiYSI6ImNsMjJmdmUzajFmeXcza3BkOXgwOWZoNW4ifQ.bFv6klpNU8XWRbEJ0zk1Dw'
    px.set_mapbox_access_token(token_name)
    fig = px.density_mapbox(df, lat = 'lat', lon = 'lon', zoom=15, mapbox_style='carto-positron', radius = 10)
    fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
    fig.show()
    write_html(fig, company_name + '_employees.html')

In [9]:
map("a79cd2fea39460c9c8f8c66673d30a34a79c59654050e441b63268dcf28186c3", "twitch")

401 available records in this search
Retrieved 100 records in batch 1
Retrieved 100 records in batch 2
Retrieved 100 records in batch 3
Retrieved 100 records in batch 4
Retrieved 1 records in batch 5
Retrieved 0 records in batch 6
Successfully recovered 401 profiles in 7 batches [39.94179892539978 seconds]
Wrote 1 lines to: 'twitch_employees.csv'
Wrote 2 lines to: 'twitch_employees.csv'
Wrote 3 lines to: 'twitch_employees.csv'
Wrote 4 lines to: 'twitch_employees.csv'
Wrote 5 lines to: 'twitch_employees.csv'
Wrote 6 lines to: 'twitch_employees.csv'
Wrote 7 lines to: 'twitch_employees.csv'
Wrote 8 lines to: 'twitch_employees.csv'
Wrote 9 lines to: 'twitch_employees.csv'
Wrote 10 lines to: 'twitch_employees.csv'
Wrote 11 lines to: 'twitch_employees.csv'
Wrote 12 lines to: 'twitch_employees.csv'
Wrote 13 lines to: 'twitch_employees.csv'
Wrote 14 lines to: 'twitch_employees.csv'
Wrote 15 lines to: 'twitch_employees.csv'
Wrote 16 lines to: 'twitch_employees.csv'
Wrote 17 lines to: 'twitch_em

AttributeError: 'float' object has no attribute 'split'